In [ ]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)  # To show all rows
pd.set_option('display.max_columns', None)  # To show all columns

In [2]:
# Read in the CSV files
googl_df = pd.read_csv("Resources/googl_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
nvda_df = pd.read_csv("Resources/nvda_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
mmm_df = pd.read_csv("Resources/mmm_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
pg_df = pd.read_csv("Resources/pg_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)

In [3]:
# Columns to drop minus the ticker prefix
column_drops = ['P/S (LTM)', 'P/FCF (LTM)', 'P/E (LTM)', 'Debt/Equity (LTM)']

# List of DataFrames and their corresponding ticker prefixes
df_ticker_pairs = [(googl_df, 'GOOGL'),(nvda_df, 'NVDA'),(mmm_df, 'MMM'),(pg_df, 'PG')]

# Loop to iterate over the list of DataFrames and their corresponding ticker prefixes
for df, ticker in df_ticker_pairs:
    # Add ticker prefix to column names and drop the columns
    columns_to_drop = [f"{ticker} {column}" for column in column_drops]
    df.drop(columns=columns_to_drop, inplace=True)
    df.drop(columns=['actual_returns'], inplace=True)

In [4]:
# Slice the "GOOGL Adj. Close" column from the DataFrame to create the "signals_googl_df" DataFrame
signals_googl_df = googl_df.loc[:,['GOOGL Adj. Close']]
signals_nvda_df = nvda_df.loc[:,['NVDA Adj. Close']]
signals_mmm_df = mmm_df.loc[:,['MMM Adj. Close']]
signals_pg_df = pg_df.loc[:,['PG Adj. Close']]

# Create lists of signal and ticker DataFrames for iteration
signal_dfs = [signals_googl_df, signals_nvda_df, signals_mmm_df, signals_pg_df]
ticker_dfs = [googl_df, nvda_df, mmm_df, pg_df]
df_pairs = [(signals_googl_df, googl_df), (signals_nvda_df, nvda_df), (signals_mmm_df, mmm_df), (signals_pg_df, pg_df)]

# Create a list of individual stock DataFrame names and columns to feed loops
ticker_data = [(signals_googl_df, "GOOGL Adj. Close"),(signals_nvda_df, "NVDA Adj. Close"),
               (signals_mmm_df, "MMM Adj. Close"),(signals_pg_df, "PG Adj. Close")]

In [5]:
# Create and Populate the "signal" column for signal DataFrames
for df in signal_dfs:
    df['Signal'] = 0.0 # Signal is the sma (10/100) signal
    df['Signal_ema'] = 0.0
    df['Signal_sma30'] = 0.0
    df['Signal_ema30'] = 0.0

In [6]:
# Generate the trading signal 0 or 1,
# where 1 is when sma_fast is greater than sma_slow or ema_fast is greater than ema_slow
# and 0 otherwise (also for sma_fast30 and ema_fast30)

for signals_df, source_df in df_pairs:
    # Apply the condition to set the 'Signal' column
    signals_df['Signal'] = np.where(source_df['sma_fast'] > source_df['sma_slow'], 1.0, 0.0) # 'Signal' is the sma (10/100) signal
    signals_df['Signal_ema'] = np.where(source_df['ema_fast'] > source_df['ema_slow'], 1.0, 0.0)
    signals_df['Signal_sma30'] = np.where(source_df['sma_fast30'] > source_df['sma_slow'], 1.0, 0.0)
    signals_df['Signal_ema30'] = np.where(source_df['ema_fast30'] > source_df['ema_slow'], 1.0, 0.0)

In [7]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points

for signals_df, source_df in df_pairs:
    signals_df['Entry/Exit'] = signals_df['Signal'].diff() # 'Entry/Exit' is the sma (10/100) signal
    signals_df['Entry/Exit_ema'] = signals_df['Signal_ema'].diff()
    signals_df['Entry/Exit_sma30'] = signals_df['Signal_sma30'].diff()
    signals_df['Entry/Exit_ema30'] = signals_df['Signal_ema30'].diff()

### SMA (10/100) Entry/Exit Plots

In [8]:
# Initialize a dictionary to store the plots for SMA (10/100) strategy for each stock
plots_sma = {}

for signals_df, source_df in df_pairs:
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in signals_df.columns if 'Adj. Close' in col][0].split()[0]

    # Plot exit positions
    exit = signals_df[signals_df['Entry/Exit'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot entry positions
    entry = signals_df[signals_df['Entry/Exit'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot security close price
    security_close = signals_df[[f'{stock_symbol} Adj. Close']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot moving averages
    moving_avgs = source_df[['sma_fast', 'sma_slow']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Create the overlay plot
    entry_exit_plot = security_close * moving_avgs * entry * exit

    # Set the title dynamically based on the stock symbol
    entry_exit_plot = entry_exit_plot.opts(
        title=f"{stock_symbol} - SMA10, SMA100, Entry and Exit Points"
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_sma[stock_symbol] = entry_exit_plot

### SMA (30/100) Entry/Exit Plots

In [9]:
# Initialize a dictionary to store the plots for SMA (30/100) strategy for each stock
plots_sma30 = {}

for signals_df, source_df in df_pairs:
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in signals_df.columns if 'Adj. Close' in col][0].split()[0]

    # Plot exit positions
    exit = signals_df[signals_df['Entry/Exit_sma30'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot entry positions
    entry = signals_df[signals_df['Entry/Exit_sma30'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot security close price
    security_close = signals_df[[f'{stock_symbol} Adj. Close']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot moving averages
    moving_avgs = source_df[['sma_fast30', 'sma_slow']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Create the overlay plot
    entry_exit_plot = security_close * moving_avgs * entry * exit

    # Set the title dynamically based on the stock symbol
    entry_exit_plot = entry_exit_plot.opts(
        title=f"{stock_symbol} - SMA30, SMA100, Entry and Exit Points"
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_sma30[stock_symbol] = entry_exit_plot

### EMA (10/100) Entry/Exit Plots

In [10]:
# Initialize a dictionary to store the plots for EMA strategy for each stock
plots_ema = {}

for signals_df, source_df in df_pairs:
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in signals_df.columns if 'Adj. Close' in col][0].split()[0]

    # Plot exit positions
    exit = signals_df[signals_df['Entry/Exit_ema'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot entry positions
    entry = signals_df[signals_df['Entry/Exit_ema'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot security close price
    security_close = signals_df[[f'{stock_symbol} Adj. Close']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot moving averages
    moving_avgs = source_df[['ema_fast', 'ema_slow']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Create the overlay plot
    entry_exit_plot = security_close * moving_avgs * entry * exit

    # Set the title dynamically based on the stock symbol
    entry_exit_plot = entry_exit_plot.opts(
        title=f"{stock_symbol} - EMA10, EMA100, Entry and Exit Points"
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_ema[stock_symbol] = entry_exit_plot

### EMA (30/100) Entry/Exit Plots

In [11]:
# Initialize a dictionary to store the plots for EMA strategy for each stock
plots_ema30 = {}

for signals_df, source_df in df_pairs:
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in signals_df.columns if 'Adj. Close' in col][0].split()[0]

    # Plot exit positions
    exit = signals_df[signals_df['Entry/Exit_ema30'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot entry positions
    entry = signals_df[signals_df['Entry/Exit_ema30'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot security close price
    security_close = signals_df[[f'{stock_symbol} Adj. Close']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot moving averages
    moving_avgs = source_df[['ema_fast30', 'ema_slow']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Create the overlay plot
    entry_exit_plot = security_close * moving_avgs * entry * exit

    # Set the title dynamically based on the stock symbol
    entry_exit_plot = entry_exit_plot.opts(
        title=f"{stock_symbol} - EMA30, EMA100, Entry and Exit Points"
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_ema30[stock_symbol] = entry_exit_plot

### Investment Capital Tracking

In [12]:
# Set initial investment capital
initial_capital = float(100000)

# Set the share size per transaction
share_size = 500

In [13]:
# Creating a position column to store the number of shares held
# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
for df in signal_dfs:
    df['Position'] = share_size * df['Signal'] # 'Position' is for the sma (10/100) signal
    df['Position_ema'] = share_size * df['Signal_ema']
    df['Position_sma30'] = share_size * df['Signal_sma30']
    df['Position_ema30'] = share_size * df['Signal_ema30']

In [14]:
# Determine the points in time where the share position is bought or sold
for df in signal_dfs:
    df['Entry/Exit Position'] = df['Position'].diff() # 'Entry/Exit Position' is for the sma (10/100) signal
    df['Entry/Exit Position_ema'] = df['Position_ema'].diff()
    df['Entry/Exit Position_sma30'] = df['Position_sma30'].diff()
    df['Entry/Exit Position_ema30'] = df['Position_ema30'].diff()

In [15]:
# Multiply the close price by the number of shares held, or the Position
for df, adj_close_col in ticker_data:
    df['Portfolio Holdings'] = df[adj_close_col] * df['Position'] # 'Portfolio Holdings' is for the sma (10/100) signal
    df['Portfolio Holdings_ema'] = df[adj_close_col] * df['Position_ema']
    df['Portfolio Holdings_sma30'] = df[adj_close_col] * df['Position_sma30']
    df['Portfolio Holdings_ema30'] = df[adj_close_col] * df['Position_ema30']

In [16]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
for df, adj_close_col in ticker_data:
    df['Portfolio Cash'] = initial_capital - (df[adj_close_col] * df['Entry/Exit Position']).cumsum() # 'Portfolio Cash' is for the sma (10/100) signal
    df['Portfolio Cash_ema'] = initial_capital - (df[adj_close_col] * df['Entry/Exit Position_ema']).cumsum()
    df['Portfolio Cash_sma30'] = initial_capital - (df[adj_close_col] * df['Entry/Exit Position_sma30']).cumsum()
    df['Portfolio Cash_ema30'] = initial_capital - (df[adj_close_col] * df['Entry/Exit Position_ema30']).cumsum()

In [17]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
for df in signal_dfs:
    df['Ticker Total'] = df['Portfolio Cash'] + df['Portfolio Holdings'] # 'Ticker Total' is for the sma (10/100) signal
    df['Ticker Total_ema'] = df['Portfolio Cash_ema'] + df['Portfolio Holdings_ema']
    df['Ticker Total_sma30'] = df['Portfolio Cash_sma30'] + df['Portfolio Holdings_sma30']
    df['Ticker Total_ema30'] = df['Portfolio Cash_ema30'] + df['Portfolio Holdings_ema30']

In [18]:
# Calculate the portfolio daily returns
for df in signal_dfs:
    df['Daily Returns'] = df['Ticker Total'].pct_change() # 'Portfolio Daily Returns' is for the sma (10/100) signal
    df['Daily Returns_ema'] = df['Ticker Total_ema'].pct_change()
    df['Daily Returns_sma30'] = df['Ticker Total_sma30'].pct_change()
    df['Daily Returns_ema30'] = df['Ticker Total_ema30'].pct_change()

In [19]:
# Calculate the portfolio cumulative returns
for df in signal_dfs:
    df['Cumulative Returns'] = (1 + df['Daily Returns']).cumprod() - 1
    df['Cumulative Returns_ema'] = (1 + df['Daily Returns_ema']).cumprod() - 1
    df['Cumulative Returns_sma30'] = (1 + df['Daily Returns_sma30']).cumprod() - 1
    df['Cumulative Returns_ema30'] = (1 + df['Daily Returns_ema30']).cumprod() - 1

In [34]:
# Write individual stock DFs with signals and capital tracking to CSV files
signals_googl_df.to_csv('Resources/googl_price_indicators.csv', index=True)
signals_nvda_df.to_csv('Resources/nvda_price_indicators.csv', index=True)
signals_mmm_df.to_csv('Resources/mmm_price_indicators.csv', index=True)
signals_pg_df.to_csv('Resources/pg_price_indicators.csv', index=True)

### SMA (10/100) Portfolio Value Plots for each ticker

In [35]:
# Initialize a dictionary to store the plots
plots_sma_portfolio_value = {}

for df in signal_dfs:
    
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]

    # Visualize exit position relative to total portfolio value
    exit = df[df['Entry/Exit'] == -1.0]['Ticker Total'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Total Position Value',
        width=1000,
        height=400
    )

    # Visualize entry position relative to total portfolio value
    entry = df[df['Entry/Exit'] == 1.0]['Ticker Total'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        ylabel='Total Position Value',
        width=1000,
        height=400
    )

    # Visualize the value of the total portfolio
    total_portfolio_value = df[['Ticker Total']].hvplot(
        line_color='lightgray',
        ylabel='Total Position Value',
        xlabel='Date',
        width=1000,
        height=400
    )

    # Overlay the plots
    portfolio_entry_exit_plot = total_portfolio_value * entry * exit

    # Set the title dynamically based on the stock symbol
    portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
        title=f"{stock_symbol} - SMA10, SMA100, Total Position Value",
        yformatter='%.0f'
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_sma_portfolio_value[stock_symbol] = portfolio_entry_exit_plot

### SMA (30/100) Portfolio Value Plots for each ticker

In [36]:
# Initialize a dictionary to store the plots
plots_sma30_portfolio_value = {}

for df in signal_dfs:
    
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]

    # Visualize exit position relative to total portfolio value
    exit = df[df['Entry/Exit_sma30'] == -1.0]['Ticker Total_sma30'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Total Position Value',
        width=1000,
        height=400
    )

    # Visualize entry position relative to total portfolio value
    entry = df[df['Entry/Exit_sma30'] == 1.0]['Ticker Total_sma30'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        ylabel='Total Position Value',
        width=1000,
        height=400
    )

    # Visualize the value of the total portfolio
    total_portfolio_value = df[['Ticker Total_sma30']].hvplot(
        line_color='lightgray',
        ylabel='Total Position Value',
        xlabel='Date',
        width=1000,
        height=400
    )

    # Overlay the plots
    portfolio_entry_exit_plot = total_portfolio_value * entry * exit

    # Set the title dynamically based on the stock symbol
    portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
        title=f"{stock_symbol} - SMA30, SMA100, Total Position Value",
        yformatter='%.0f'
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_sma30_portfolio_value[stock_symbol] = portfolio_entry_exit_plot

### EMA (10/100) Portfolio Value Plots for each ticker

In [37]:
# Initialize a dictionary to store the plots
plots_ema_portfolio_value = {}

for df in signal_dfs:
    
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]

    # Visualize exit position relative to total portfolio value
    exit = df[df['Entry/Exit_ema'] == -1.0]['Ticker Total_ema'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Total Position Value',
        width=1000,
        height=400
    )

    # Visualize entry position relative to total portfolio value
    entry = df[df['Entry/Exit_ema'] == 1.0]['Ticker Total_ema'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        ylabel='Total Position Value',
        width=1000,
        height=400
    )

    # Visualize the value of the total portfolio
    total_portfolio_value = df[['Ticker Total_ema']].hvplot(
        line_color='lightgray',
        ylabel='Total Position Value',
        xlabel='Date',
        width=1000,
        height=400
    )

    # Overlay the plots
    portfolio_entry_exit_plot = total_portfolio_value * entry * exit

    # Set the title dynamically based on the stock symbol
    portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
        title=f"{stock_symbol} - EMA10, EMA100, Total Position Value",
        yformatter='%.0f'
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_ema_portfolio_value[stock_symbol] = portfolio_entry_exit_plot

### EMA (30/100) Portfolio Value Plots for each ticker

In [38]:
# Initialize a dictionary to store the plots
plots_ema30_portfolio_value = {}

for df in signal_dfs:
    
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]

    # Visualize exit position relative to total portfolio value
    exit = df[df['Entry/Exit_ema30'] == -1.0]['Ticker Total_ema30'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Total Position Value',
        width=1000,
        height=400
    )

    # Visualize entry position relative to total portfolio value
    entry = df[df['Entry/Exit_ema30'] == 1.0]['Ticker Total_ema30'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        ylabel='Total Position Value',
        width=1000,
        height=400
    )

    # Visualize the value of the total portfolio
    total_portfolio_value = df[['Ticker Total_ema30']].hvplot(
        line_color='lightgray',
        ylabel='Total Position Value',
        xlabel='Date',
        width=1000,
        height=400
    )

    # Overlay the plots
    portfolio_entry_exit_plot = total_portfolio_value * entry * exit

    # Set the title dynamically based on the stock symbol
    portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
        title=f"{stock_symbol} - EMA30, EMA100, Total Position Value",
        yformatter='%.0f'
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_ema30_portfolio_value[stock_symbol] = portfolio_entry_exit_plot

### Display All Plots

In [39]:
# Display the SMA and EMA plots for individual tickers
display(plots_sma['GOOGL'])
display(plots_sma30['GOOGL'])
display(plots_ema['GOOGL'])
display(plots_ema30['GOOGL'])
display(plots_sma_portfolio_value['GOOGL'])
display(plots_sma30_portfolio_value['GOOGL'])
display(plots_ema_portfolio_value['GOOGL'])
display(plots_ema30_portfolio_value['GOOGL'])

display(plots_sma['NVDA'])
display(plots_sma30['NVDA'])
display(plots_ema['NVDA'])
display(plots_ema30['NVDA'])
display(plots_sma_portfolio_value['NVDA'])
display(plots_sma30_portfolio_value['NVDA'])
display(plots_ema_portfolio_value['NVDA'])
display(plots_ema30_portfolio_value['NVDA'])

display(plots_sma['MMM'])
display(plots_sma30['MMM'])
display(plots_ema['MMM'])
display(plots_ema30['MMM'])
display(plots_sma_portfolio_value['MMM'])
display(plots_sma30_portfolio_value['MMM'])
display(plots_ema_portfolio_value['MMM'])
display(plots_ema30_portfolio_value['MMM'])

display(plots_sma['PG'])
display(plots_sma30['PG'])
display(plots_ema['PG'])
display(plots_ema30['PG'])
display(plots_sma_portfolio_value['PG'])
display(plots_sma30_portfolio_value['PG'])
display(plots_ema_portfolio_value['PG'])
display(plots_ema30_portfolio_value['PG'])

:Overlay
   .Curve.I                              :Curve   [Date]   (GOOGL Adj. Close)
   .NdOverlay.I                          :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.GOOGL_Adj_full_stop_Close.I  :Scatter   [Date]   (GOOGL Adj. Close)
   .Scatter.GOOGL_Adj_full_stop_Close.II :Scatter   [Date]   (GOOGL Adj. Close)

:Overlay
   .Curve.I                              :Curve   [Date]   (GOOGL Adj. Close)
   .NdOverlay.I                          :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.GOOGL_Adj_full_stop_Close.I  :Scatter   [Date]   (GOOGL Adj. Close)
   .Scatter.GOOGL_Adj_full_stop_Close.II :Scatter   [Date]   (GOOGL Adj. Close)

:Overlay
   .Curve.I                              :Curve   [Date]   (GOOGL Adj. Close)
   .NdOverlay.I                          :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.GOOGL_Adj_full_stop_Close.I  :Scatter   [Date]   (GOOGL Adj. Close)
   .Scatter.GOOGL_Adj_full_stop_Close.II :Scatter   [Date]   (GOOGL Adj. Close)

:Overlay
   .Curve.I                              :Curve   [Date]   (GOOGL Adj. Close)
   .NdOverlay.I                          :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.GOOGL_Adj_full_stop_Close.I  :Scatter   [Date]   (GOOGL Adj. Close)
   .Scatter.GOOGL_Adj_full_stop_Close.II :Scatter   [Date]   (GOOGL Adj. Close)

:Overlay
   .Curve.I                 :Curve   [Date]   (Ticker Total)
   .Scatter.Ticker_Total.I  :Scatter   [Date]   (Ticker Total)
   .Scatter.Ticker_Total.II :Scatter   [Date]   (Ticker Total)

...

:Overlay
   .Curve.I                           :Curve   [Date]   (PG Adj. Close)
   .NdOverlay.I                       :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.PG_Adj_full_stop_Close.I  :Scatter   [Date]   (PG Adj. Close)
   .Scatter.PG_Adj_full_stop_Close.II :Scatter   [Date]   (PG Adj. Close)

:Overlay
   .Curve.I                 :Curve   [Date]   (Ticker Total)
   .Scatter.Ticker_Total.I  :Scatter   [Date]   (Ticker Total)
   .Scatter.Ticker_Total.II :Scatter   [Date]   (Ticker Total)

:Overlay
   .Curve.I                       :Curve   [Date]   (Ticker Total_sma30)
   .Scatter.Ticker_Total_sma30.I  :Scatter   [Date]   (Ticker Total_sma30)
   .Scatter.Ticker_Total_sma30.II :Scatter   [Date]   (Ticker Total_sma30)

:Overlay
   .Curve.I                     :Curve   [Date]   (Ticker Total_ema)
   .Scatter.Ticker_Total_ema.I  :Scatter   [Date]   (Ticker Total_ema)
   .Scatter.Ticker_Total_ema.II :Scatter   [Date]   (Ticker Total_ema)

:Overlay
   .Curve.I                       :Curve   [Date]   (Ticker Total_ema30)
   .Scatter.Ticker_Total_ema30.I  :Scatter   [Date]   (Ticker Total_ema30)
   .Scatter.Ticker_Total_ema30.II :Scatter   [Date]   (Ticker Total_ema30)